::::
:::{thebe-button}
:::
::::

# Find tracks

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm {cite}`allanTrackpy2018,crockerMethodsDigitalVideo1996`.


In [ ]:
from collections.abc import Iterable

from devtools import pprint
from more_itertools import one, only
from numpy import diff, linalg
from pandas import DataFrame, read_hdf

from boilercv.data import FRAME, VIDEO
from boilercv.images import scale_bool
from boilercv_docs.nbs import init
from boilercv_pipeline.models.column import Col
from boilercv_pipeline.models.deps import get_slices
from boilercv_pipeline.models.df import GBC
from boilercv_pipeline.models.subcool import const
from boilercv_pipeline.models.unit import M_TO_MM
from boilercv_pipeline.sets import get_dataset, get_dataset2
from boilercv_pipeline.stages.find_tracks import FindTracks as Params

PARAMS = None

SEARCH_RANGE = 30
"""Pixel range to search for the next bubble."""
MEMORY = 5
"""Frames to remember a bubble."""

PX_PER_M = 20997.3753

In [ ]:
if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    params = Params(
        context=init(),
        include_patterns=const.nb_include_patterns,
        slicer_patterns=const.nb_slicer_patterns,
    )
params.format.set_display_options()
data = params.data
C = params.cols
context = params.context
objects = DataFrame(read_hdf(one(params.objects)))
slices = get_slices(one(params.filled_slicers))
frames = slices.get(FRAME, slice(None))
filled = scale_bool(get_dataset2(one(params.filled), slices=slices)[VIDEO])
dfs = only(params.dfs)


def preview(
    df: DataFrame, cols: Iterable[Col] | None = None, index: Col | None = None
) -> DataFrame:
    """Preview a dataframe in the notebook."""
    df = params.format.preview(
        cols=cols,
        df=df,
        index=index,
        f=lambda df: df.groupby(C.frame(), **GBC).head(3).head(6),
    )
    return df


pprint(params)

## Find bubbles in each frame and link them

Detect individual bubbles in each frame, and then link detections across frames by application of the Crocker-Grier tracking algorithm, which takes into account centroid proximity and expected positions {cite}`crockerMethodsDigitalVideo1996`.

Initial and lifetime characteristics of long-lived bubbles are shown in **Table&NonBreakingSpace;1**. All bubbles departing the surface have an initial depth, $y$, close to the actual boiling surface, and a bimodal distribution in initial $x$, close to active nucleation sites. This information is used to determine surface and departure $y$ thresholds for alignment of bubble departures.


In [ ]:
objects: DataFrame = read_hdf(
    (params.deps.objects / f"objects_{path_time}").with_suffix(".h5")
)  # pyright: ignore[reportAssignmentType]

In [ ]:
path_time = p.time.replace(":", "-")
video = get_dataset(p.time.replace(":", "-"))["video"]
frametime = diff(video.time.values).mean()
objects: DataFrame = read_hdf((OBJECTS / f"objects_{path_time}").with_suffix(".h5"))  # pyright: ignore[reportAssignmentType]
tracks = (
    link(f=objects, search_range=SEARCH_RANGE, memory=MEMORY)
    .rename(columns={"x": "x_px", "y": "y_px"})
    .assign(
        frame_lifetime=(
            lambda df: df.groupby("particle", **GBC)["frame"].transform("count")
        )
    )
    .sort_values(["frame_lifetime", "particle", "frame"], ascending=[False, True, True])
    .assign(
        bubble=(lambda df: df.groupby("particle", **GBC).ngroup()),
        dy_px=lambda df: df.groupby("bubble", **GBC)[["y_px"]].diff().fillna(0),
        dx_px=lambda df: df.groupby("bubble", **GBC)[["x_px"]].diff().fillna(0),
        y=lambda df: df["y_px"] / PX_PER_M,
        x=lambda df: df["x_px"] / PX_PER_M,
        dy=lambda df: df["dy_px"] / PX_PER_M / frametime,
        dx=lambda df: df["dx_px"] / PX_PER_M / frametime,
        diameter=lambda df: df["diameter_px"] / PX_PER_M,
        radius_of_gyration=lambda df: df["radius_of_gyration_px"] / PX_PER_M,
        distance=lambda df: linalg.norm(df[["dx", "dy"]].abs(), axis=1),
        time=lambda df: video.sel(frame=df["frame"].values)["time"],
        lifetime=lambda df: df["frame_lifetime"] * frametime,
    )
    .drop(columns=["particle"])
)

with style_df(
    tracks.groupby("bubble", **GBC)
    .head(1)
    .set_index("bubble")
    .head(16)
    .pipe(
        transform_cols,
        cols=[
            Col("lifetime", "Lifetime", "s"),
            Col("time", r"$t_0$", "s"),
            Col("diameter", r"$d_{b0}$", **M_TO_MM),
            Col("y", r"$y_{b0}$", **M_TO_MM),
            Col("x", r"$x_{b0}$", **M_TO_MM),
        ],
    )
) as style:
    display_dataframe_with_math(style.background_gradient())

**Table&NonBreakingSpace;1**: Selected properties of long-lived bubbles
Bubbles are identified by a unique particle number. Their lifetime, the time of their first appearance, their initial diameter and elevation, and lifetime histograms of selected characteristics are shown.
